In [71]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report, confusion_matrix
from skimage.feature import hog
from sklearn.decomposition import PCA
import pickle

In [72]:
# Étape 1 : Collecte de données (CK+ dataset)
data_directory = 'extract/train_10000'
emotions = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]

In [73]:
# Étape 2 : Prétraitement des données
def preprocess_data(data_directory):
    images = []
    labels = []

    for emotion in emotions:
        emotion_path = os.path.join(data_directory, emotion)
        for image_name in os.listdir(emotion_path):
            image_path = os.path.join(emotion_path, image_name)
            image = cv2.imread(image_path, 0)  # Charger l'image en niveaux de gris
            image = cv2.resize(image, (48, 48))  # Redimensionner l'image
            images.append(image)
            labels.append(emotions.index(emotion))  # Étiqueter l'image avec l'indice de l'émotion

    return np.array(images), np.array(labels)

images, labels = preprocess_data(data_directory)

In [74]:
# Étape 3 : Extraction des caractéristiques (HOG)
def extract_features(images):
    features = []

    for image in images:
        # Appliquer le descripteur HOG
        hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        features.append(hog_features)

    return np.array(features)

features = extract_features(images)


In [75]:
# Étape 4 : Réduction de dimension avec PCA
pca = PCA(n_components=60)
features_pca = pca.fit_transform(features)

In [76]:
# Étape 5 : Construction du modèle
def train_model(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    model = RandomForestClassifier()  # Utilisation d'un modèle de forêt aléatoire
    model.fit(X_train, y_train)
    return model
model = train_model(features_pca, labels)

In [77]:
# Étape 6 : Évaluation du modèle
def evaluate_model(model, features, labels):
    features_pca = pca.transform(features)
    predictions = model.predict(features_pca)
    accuracy = accuracy_score(labels, predictions)
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    report = classification_report(labels, predictions, target_names=emotions)
    confusion = confusion_matrix(labels, predictions)
    return accuracy, recall, f1, report, confusion

In [78]:

accuracy, recall, f1, report, confusion = evaluate_model(model, features, labels)

print("Précision :", accuracy)
print("Rappel :", recall)
print("Score F1 :", f1)
print("Rapport de classification :\n", report)
print("Matrice de confusion :\n", confusion)

Précision : 0.9198865345768685
Rappel : 0.9198865345768685
Score F1 : 0.9197780334864748
Rapport de classification :
               precision    recall  f1-score   support

       anger       0.90      0.90      0.90     10000
     disgust       0.92      0.96      0.94      9802
        fear       0.94      0.89      0.92     10000
   happiness       0.92      0.92      0.92     10000
     neutral       0.91      0.91      0.91     10000
     sadness       0.92      0.91      0.91     10000
    surprise       0.94      0.96      0.95      9999

    accuracy                           0.92     69801
   macro avg       0.92      0.92      0.92     69801
weighted avg       0.92      0.92      0.92     69801

Matrice de confusion :
 [[8973  410  135  126  133  148   75]
 [ 213 9366   61   55   37   51   19]
 [ 261  197 8939  108  145  164  186]
 [ 197   64   81 9153  238  156  111]
 [ 159   57  109  213 9132  176  154]
 [ 167  103  135  183  267 9058   87]
 [  50   28   69   93  100   71 9

In [79]:

# Chargement des nouvelles données de test
new_test_data_directory = 'extract/data'
new_test_images, new_test_labels = preprocess_data(new_test_data_directory)

In [80]:
# Extraction des caractéristiques pour les nouvelles données de test
new_test_features = extract_features(new_test_images)

In [81]:
# Évaluation du modèle sur les nouvelles données de test
new_test_features_pca = pca.transform(new_test_features)
new_test_predictions = model.predict(new_test_features_pca)
new_test_accuracy = accuracy_score(new_test_labels, new_test_predictions)
new_test_recall = recall_score(new_test_labels, new_test_predictions, average='weighted')
new_test_f1 = f1_score(new_test_labels, new_test_predictions, average='weighted')
new_test_report = classification_report(new_test_labels, new_test_predictions, target_names=emotions)
new_test_confusion = confusion_matrix(new_test_labels, new_test_predictions)

print("Précision sur les nouvelles données de test :", new_test_accuracy)
print("Rappel sur les nouvelles données de test :", new_test_recall)
print("Score F1 sur les nouvelles données de test :", new_test_f1)
print("Rapport de classification pour les nouvelles données de test :\n", new_test_report)
print("Matrice de confusion pour les nouvelles données de test :\n", new_test_confusion)


Précision sur les nouvelles données de test : 0.7514285714285714
Rappel sur les nouvelles données de test : 0.7514285714285714
Score F1 sur les nouvelles données de test : 0.746664717706357
Rapport de classification pour les nouvelles données de test :
               precision    recall  f1-score   support

       anger       0.64      0.78      0.70      2000
     disgust       0.72      0.92      0.81      2000
        fear       0.73      0.50      0.60      2000
   happiness       0.76      0.63      0.69      2000
     neutral       0.72      0.75      0.73      2000
     sadness       0.82      0.76      0.79      2000
    surprise       0.89      0.91      0.90      2000

    accuracy                           0.75     14000
   macro avg       0.76      0.75      0.75     14000
weighted avg       0.76      0.75      0.75     14000

Matrice de confusion pour les nouvelles données de test :
 [[1564  112   69   61  103   52   39]
 [  83 1846   15   23   18   13    2]
 [ 330  276 10

In [82]:
# Sauvegarde du modèle RandomForestClassifier et du modèle PCA dans des fichiers
model_filename = 'emotion_model.pkl'
pca_filename = 'pca_model1.pkl'
pickle.dump(model, open(model_filename, 'wb'))
pickle.dump(pca, open(pca_filename, 'wb'))

print("Le modèle RandomForestClassifier a été sauvegardé dans le fichier", model_filename)
print("Le modèle PCA a été sauvegardé dans le fichier", pca_filename)

Le modèle RandomForestClassifier a été sauvegardé dans le fichier emotion_model.pkl
Le modèle PCA a été sauvegardé dans le fichier pca_model1.pkl


In [86]:
from sklearn.metrics import cohen_kappa_score


def save_element(x, name="element", binary=True, ext=".pkl"):
    if binary:
        mode = 'wb'
        encod = '_bin'
    else:
        encod = ""
        mode = 'w'
    file = open(name + encod + ext, mode)
    pickle.dump(x, file)
    file.close()
    print("File " + name + " saved")

test_metrics = {
    'test_accuracy': accuracy_score(new_test_labels, new_test_predictions),
    'test_report': classification_report(new_test_labels, new_test_predictions, output_dict=False, target_names=emotions),
    'test_report_dict': classification_report(new_test_labels, new_test_predictions, output_dict=True, target_names=emotions),
    'test_cf_matrix': confusion_matrix(new_test_labels, new_test_predictions),
    'test_kappa': cohen_kappa_score(new_test_labels, new_test_predictions)
}

save_element(test_metrics, "test_performance_metrics")

File test_performance_metrics saved
